In [4]:
function enrich_generic3(ϕ, ψ⃗; cutoff=1e-15, kwargs...)

"""
  Given spec from the eigen function, to extract its information use the 
  following functions:

  eigs(spec) returns the spectrum
  truncerror(spec) returns the truncation error
"""  
  Nₘₚₛ = length(ψ⃗) ##number of MPS

  @assert all(ψᵢ -> length(ψ⃗[1]) == length(ψᵢ), ψ⃗) ##check that all MPS inputs are of the same length

  N = length(ψ⃗[1]) 

  ψ⃗ = copy.(ψ⃗)
    
  ###Isn't this already a vector of MPS's?  
  ψ⃗ = convert.(MPS, ψ⃗)

  s = siteinds(ψ⃗[1])

  ##makes the orthogonality centre for each MPS to be at site N  
  ψ⃗ = orthogonalize.(ψ⃗, N)
  ϕ = orthogonalize!(ϕ, N)
  
  ##storage MPS
  phi = deepcopy(ϕ)
  
  ρϕ = prime(ϕ[N], s[N]) * dag(ϕ[N])
  ρ⃗ₙ = [prime(ψᵢ[N], s[N]) * dag(ψᵢ[N]) for ψᵢ in ψ⃗]
  ρₙ = sum(ρ⃗ₙ)
 
  """
  Is this needed?
  """
  ρₙ /=tr(ρₙ)

#   # Maximum theoretical link dimensions
 
  Cϕprev = ϕ[N]
  C⃗ₙ = last.(ψ⃗)
  
 
  for n in reverse(2:N)
     """
    In the paper they propose to do this step with no truncation. At the very
    least this cutoff should be a function parameter.
    """    
    
    left_inds = linkind(ϕ,n-1)
   
         #Diagonalize primary state ψ's density matrix    
    U,S,Vϕ,spec = svd(Cϕprev,left_inds; 
      lefttags = tags(linkind(ϕ, n - 1)),
      righttags = tags(linkind(ϕ, n - 1)))   

    x = dim(inds(S)[1])
    @assert(x == dim(linkind(ϕ, n - 1)))
    r = uniqueinds(Vϕ, S) # Indices of density matrix
    lϕ = commonind(S, Vϕ) # Inner link index from density matrix diagonalization
        

    # Compute the theoretical maximum bond dimension that the enriched state cannot exceeded: 
    abs_maxdim = bipart_maxdim(s,n - 1) - dim(lϕ)        
    # Compute the number of eigenvectors of ɸ's projected density matrix to retain:
    Kry_linkdim_vec = [dim(linkind(ψᵢ, n - 1)) for ψᵢ in ψ⃗]
        
   
    ω_maxdim = min(sum(Kry_linkdim_vec),abs_maxdim)
    
    if ω_maxdim !== 0


        # Construct identity matrix
        ID = 1
        rdim = 1
        for iv in r
          IDv = ITensor(dag(iv)', iv);
          rdim *= dim(iv)
          for i in 1:dim(iv)
            IDv[iv' => i, iv => i] = 1.0
          end      
          ID = ID*IDv
        end   


        P = ID - prime(Vϕ, r)*dag(Vϕ) # Projector on to null-space of ρψ   
      
        C = combiner(r) # Combiner for indices
        # Check that P is non-zero   
        if abs(tr(matrix(C'*P*dag(C)))) > 1e-10    


            Dp, Vp, spec_P = eigen(
                  P, r', r;
                  ishermitian=true,
                  tags="P space",
                  cutoff=1e-1,
                  maxdim=rdim-dim(lϕ),             ###potentially wrong
                  kwargs...,
                )

            lp = commonind(Dp,Vp)
           
            ##constructing VpρₙVp
            VpρₙVp = Vp*ρₙ        
            VpρₙVp = VpρₙVp*dag(Vp')
            chkP = abs(tr(matrix(VpρₙVp))) ##chkP

        else
            chkP = 0    
        end
    else
        chkP = 0
    end
  
    if chkP >1e-15
        

        Dₙ, Vₙ, spec = eigen(VpρₙVp, lp', lp;
          ishermitian=true,
          tags=tags(linkind(ψ⃗[1], n - 1)),
          cutoff=cutoff,
          maxdim=ω_maxdim,            
          kwargs...,
        )
        
        Vₙ = Vp*Vₙ

        lₙ₋₁ = commonind(Dₙ, Vₙ)
 
        # Construct the direct sum isometry 
        V, lnew = directsum(Vϕ => lϕ, Vₙ => lₙ₋₁; tags = tags(linkind(ϕ, n - 1)))
    else
         V = Vϕ
         lnew = lϕ
            
    end
    @assert dim(linkind(ϕ, n - 1)) - dim(lϕ) <=0
    # Update the enriched state
    phi[n] = V
  
 
    # Compute the new density matrix for the ancillary states
    C⃗ₙ₋₁ = [ψ⃗[i][n - 1] * C⃗ₙ[i] * dag(V) for i in 1:Nₘₚₛ]   
    C⃗ₙ₋₁′ = [prime(Cₙ₋₁, (s[n - 1], lnew)) for Cₙ₋₁ in C⃗ₙ₋₁]    
    ρ⃗ₙ₋₁ = C⃗ₙ₋₁′ .* dag.(C⃗ₙ₋₁)
    ρₙ₋₁ = sum(ρ⃗ₙ₋₁)
    
    # compute the density matrix for the real state    
    Cϕ = ϕ[n - 1] * Cϕprev * dag(V)
    Cϕd = prime(Cϕ, (s[n - 1], lnew))
    ρϕ = Cϕd * dag(Cϕ) 
        

    Cϕprev = Cϕ
    C⃗ₙ = C⃗ₙ₋₁
    ρₙ = ρₙ₋₁
  
  end
    
    
    phi[1] = Cϕprev
    phi[1] = phi[1]/norm(phi)
 
  return phi
end

function bipart_maxdim(s,n)
# Compute the theoretical maximum link dimension for an orthogonalised MPS
# for the bipartition [1,...,n][n+1,...,N]
    left_maxdim = 1
    for k=1:n
        left_maxdim *= dim(s[k])
    end
    right_maxdim = 1
    for k=(n+1):length(s)
        right_maxdim *= dim(s[k])
    end
    return min(left_maxdim,right_maxdim)
end;




function Krylov_states(H,ψ,k,τ,s)
    """
    POTENTIAL PROBLEM: The Krylov states aren't canonical so don't satisfy the constraint. orthogonalizing could
    potentially sort this.
    """
    
    ##Create the first k Krylov states
    Id = MPO(s,"Id")
    Kry_op = Id-im*τ*H
    list = []
    term = copy(ψ)
   
   for i =1:k-1
        term = noprime(Kry_op*term)
        term = term/norm(term)
        push!(list,term)
    end

    return list
end


function Krylov_linkdims(Krylov)
    """
Determining whether a Krylov state has the dimensions of lower Krylov states within it.
We have a list of vectors, and we want to see if the 1st vector has the highest entries for every entry.
I want the output to be a vector of length of linkdims, where each entry denotes which Krylov vector has the maximum dimension.
"""
    x = linkdims(Krylov[1])
    dim1 = length(x)
    dim2 = length(Krylov)
    output = zeros(length(x))
    stuff = zeros(dim1,dim2)
    for i =1:dim1
        for j =1:dim2
            stuff[i,j] = linkdims(Krylov[j])[i]
        end
        vec = stuff[i,:]
        term = Int(argmax(vec)) 
        output[i] = term
        if term != dim2
           test = vec[term] -vec[dim2]
           if test<=0
               output[i] = dim2
           end
        end
    end
    return output
end


function linkdims(ψ)
    # Helpful function for outputing the link dimension profile of an MPS

  """
  Isn't working properly.  
  """
  linkdims = zeros(length(ψ)-1,1)
  for b in eachindex(ψ)[1:(end - 1)]
    l = linkind(ψ, b)
    linkdims[b] = isnothing(l) ? 1 : dim(l)
  end
  return linkdims
end


linkdims (generic function with 1 method)